In [1]:
import csv
import pandas as pd
import numpy as np
import gmaps
gmaps.configure(api_key='YOUR_API_KEY')

### Import the data
Use the powerful packege [Instagram Scraper](https://github.com/rarcega/instagram-scraper) to scrape data.    
Note that the scraper cannot get the latitude & longitude of the post, we can only find location_id.      
Therefore, I use beautifulsoup to scrape the latitude & longitude from the page of location_id, in this notebook, the part of data cleaning will not be shown. 

In [2]:
df_4foodie = pd.read_csv('YOUR_FILE_PATH',sep='\t', encoding='utf-8')
df_4foodie.drop(['Unnamed: 0'],axis = 1, inplace = True)
df_4foodie.head()

,display_url,like,content,comment,is_video,location,shortcode,tags,timestamp,username,location_id,city,country_code,time,hour,len(tags),weekday,lat,long,video_view_count
0,https://scontent-hkg3-2.cdninstagram.com/vp/d2...,4486,"📍️Tokyo, Japan\n小枝 / 塩バターキャラメル / ¥216(税込)\n美味程...",14,False,"Tokyo, Japan",Bt-Jr-ehpjo,"['東京', '便利商店', '塩キャラメル', '4foodie', 'グルメ', '全家...",1550381192,4foodie,7.226110e+06,NaN,JP,2019-02-17 13:26:32,13,16,Sunday,35.680000,139.770000,NaN
1,https://scontent-hkg3-2.cdninstagram.com/vp/a9...,5585,"📍Taipei, Taiwan\nBonheur Cookie小小幸福手作烘培食事 @bon...",20,False,NaN,Bt8OQLHBml6,"['甜點', '蛋糕', '台北美食', '草莓', '4foodie', '奶油餅', '...",1550316477,4foodie,0.000000e+00,NaN,NaN,2019-02-16 19:27:57,19,16,Saturday,NaN,NaN,NaN
2,https://scontent-hkg3-2.cdninstagram.com/vp/4c...,3642,"📍San Gabriel, California \n吴銘火鍋 Wu Ming Hot Po...",29,False,Wu Ming Hotpot,Bt74-6lhOh5,"['加州美食', '洛杉磯美食', '鴛鴦鍋', '4foodie加州', 'wumingh...",1550305326,4foodie,2.220091e+14,"San Gabriel, California",US,2019-02-16 16:22:06,16,14,Saturday,34.080580,-118.093010,NaN
3,https://scontent-hkg3-2.cdninstagram.com/vp/f9...,4562,感謝 @amos0716 的分享🙏🏻\n🔹標記 #4foodieforfoodie 就有機會...,39,False,駁二藝術特區 The Pier-2 Art Center,Bt5kUlihaRr,"['4foodie', '正面奶酥', '4foodie高雄', '麵包', '4foodi...",1550227384,4foodie,8.011200e+04,"Kaohsiung, Taiwan",TW,2019-02-15 18:43:04,18,9,Friday,22.620228,120.281517,NaN
4,https://scontent-hkg3-2.cdninstagram.com/vp/ed...,7895,感謝 @missbestie_ 的分享🙏🏻\n🔹標記 #4foodieforfoodie 就...,59,False,Little babee 小貝bee脆皮雞蛋糕,Bt41YjNhC8g,"['高雄小吃', '甜點', '雞蛋糕', 'littlebabee', '新興區美食', ...",1550202775,4foodie,2.043458e+14,"Kaohsiung, Taiwan",TW,2019-02-15 11:52:55,11,13,Friday,22.633450,120.309240,NaN


In [3]:
print('There are ' + str(int(df_4foodie.describe().at['count', 'lat'])) + ' data we can draw on google map')

There are 2416 data we can draw on google map


### Heatmap

In [4]:
locations = []
for i in range(len(df_4foodie)):
    if pd.isna(df_4foodie.lat[i]) == False:
        locations.append((float(df_4foodie.lat[i]), float(df_4foodie.long[i])))
taipei = (25.04776, 121.53185)
fig = gmaps.figure(center=taipei, zoom_level=13)
heatmap_layer = gmaps.heatmap_layer(
    locations, point_radius = 20 , max_intensity=30)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Marker map

Prepar for embedding the post in the info box of marker map

In [5]:
form_info_box = '''
<iframe frameborder="0" src="https://instagram.com/p/{shortcode}/embed/" width="100%" height="500" scrolling="no" allowtransparency="true" class="note-video-clip">
</iframe>
'''

In [6]:
info_box_lst = []
locations = []
#Since too many markers will diminish the readability and cause Jupyter Notebook lag, I use 100 instead of all data.
for i in range(100): 
    if pd.isna(df_4foodie.lat[i]) == False:
        locations.append((float(df_4foodie.lat[i]), float(df_4foodie.long[i])))
        info_box_lst.append(form_info_box.format(shortcode = df_4foodie.at[i,"shortcode"]))
taipei = (25.04776, 121.53185)
fig2 = gmaps.figure(center=taipei, zoom_level=12)
marker_layer = gmaps.marker_layer(locations, info_box_content=info_box_lst)
fig2.add_layer(marker_layer)
fig2

Figure(layout=FigureLayout(height='420px'))

You can also put other information in the info box using HTML form,  
but I think the interface is good enough if we just embed the post